# MSDS3436 Final Project
<h3>Process Source Dataset</h3>

2022WI_MSDS_436-DL_SEC55 Analytics Systems Engineering

Mark Stockwell | Wesley Ng

Please follow the steps below to download and use kaggle data within Google Colab:

1. Go to your account, Scroll to API section and Click Expire API Token to remove previous tokens
2. Click on Create New API Token - It will download kaggle.json file on your machine.
3. Go to your Google Colab project file and run the following commands, the files.upload() command will prompt you for location of json file.

```
! pip install -q kaggle
from google.colab import files
files.upload()
```
Choose the kaggle.json file that you downloaded, then run the following commands to copy the file to standard colab runtime directory:
```
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
```
Change the permissions of the file.
```
! chmod 600 ~/.kaggle/kaggle.json
```
That's all ! You can check if everything's okay by running this command.
```
! kaggle datasets list
```

Download Data

```
! kaggle competitions download -c 'name-of-competition'
```
Use unzip command to unzip the data:
For example,
Create a directory named train, unzip train data there
```
! mkdir train
! unzip train.zip -d train
```

# Kaggle Setup

Install the package and update the json key for downloads.

In [ ]:
! pip3 install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# all ok? this command should work
! kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                            title                                              size  lastUpdated          downloadCount  
-------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
yasserh/wine-quality-dataset                                   Wine Quality Dataset                               21KB  2022-01-15 19:15:11           5052  
prasertk/netflix-subscription-price-in-different-countries     Netflix subscription fee in different countries     3KB  2022-01-15 07:06:09           4458  
prasertk/the-best-cities-for-a-workation                       The Best Cities for a Workation                     6KB  2022-02-16 11:46:28            419  
majyhain/height-of-male-and-female-by-country-2022             Height of Male and Female by Country 2022           4KB  2022-02-02 00:40:19           1864  
sanjeetsinghnaik/top-100

## Download the Facemask images

Install the Open Datasets package and download files.

Open Datasets package unzips and creates directories.

https://pypi.org/project/opendatasets/

Images from https://www.kaggle.com/pranavsingaraju/facemask-detection-dataset-20000-images 

In [ ]:
!pip3 install opendatasets
import datetime
import opendatasets as od
print("starting download",datetime.datetime.now())
od.download("https://www.kaggle.com/pranavsingaraju/facemask-detection-dataset-20000-images")
print("finished download",datetime.datetime.now())

starting download 2022-02-21 01:52:04.296255


100%|██████████| 604M/604M [00:13<00:00, 48.2MB/s]



finished download 2022-02-21 01:52:24.664768


# AWS Setup

You will need to upload two files with no extension, "credentials" and "config".

The credentials file has your API key, config is preferred region

Put the following in each, change values to yours:
<hr>
credentials

```
[default]
aws_access_key_id = AKIAXN6R7JXYxxxxxxxx 
aws_secret_access_key = 1prI4W8J254zR5/ishEilmqskaZLOnazxxxxxxxx
```
<hr>
config

```
[default]
region = us-east-1
```

In [ ]:
!apt-get update
!pip3 install awscli
!pip3 install boto3==1.18
!pip3 install sagemaker

import boto3
from google.colab import files
files.upload()
! mkdir ~/.aws
! cp credentials ~/.aws/
! cp config ~/.aws/
! chmod 600 ~/.aws/credentials
! chmod 600 ~/.aws/config

# all ok? this command should work
comprehend = boto3.client(service_name='comprehend', region_name="us-east-1")
text = "Machine learning will automate jobs that most people thought could only be done by people." #~Dave Waters
print("text to perform sentiment analysis: \n",text)
comprehend.detect_sentiment(Text=text, LanguageCode='en')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

KeyboardInterrupt: ignored

## Upload files to S3

In [ ]:
from re import sub
import boto3
import os
 
def upload_files(path,bucket):
    session = boto3.Session()
    s3 = session.resource('s3')
    bucket = s3.Bucket(bucket)
    filecount = 0
    for subdir, dirs, files in os.walk(path):
      print('uploading dir',subdir)
      for file in files:
        full_path = os.path.join(subdir, file)
        with open(full_path, 'rb') as data:
            bucket.put_object(Key=full_path[len(path)+1:], Body=data)
        filecount +=1
      print('  file count:',filecount)

# call function
upload_files('/content/facemask-detection-dataset-20000-images','511004593648-msds436')

In [ ]:
# split into test/train
from re import sub
import boto3
import os
import random

bucket_name = '511004593648-msds436'

def copy_s3_files(src_obj, tgt_obj):
    session = boto3.Session()
    s3 = session.resource('s3')
    bucket = s3.Bucket(bucket_name)
    copy_source = {
        'Bucket': bucket_name,
        'Key': src_obj
    }
    bucket.copy(copy_source, tgt_obj)

# Randomly move files 
train_size = 100
test_size = 20
train_range = random.sample(range(10000),train_size)
test_range = random.sample(range(10000),test_size)
for n in train_range:
  file_name = "seed" + str(n).zfill(4) + ".png"
  f_with = "new_with_mask/with-mask-default-mask-" + file_name
  f_without = "new_without_mask/" + file_name
  copy_s3_files(f_without, f'train/{file_name}')
  copy_s3_files(f_with, f'train/{file_name}')
for n in test_range:
  file_name = "seed" + str(n).zfill(4) + ".png"
  f_with = "new_with_mask/with-mask-default-mask-" + file_name
  f_without = "new_without_mask/" + file_name
  copy_s3_files(f_without, f'test/{file_name}')
  copy_s3_files(f_with, f'test/{file_name}')

## Configure Sagemaker

In [ ]:
%%time
import boto3
import re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()

training_image = get_image_uri(boto3.Session().region_name, "image-classification")